In [24]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy.optimize import minimize

In [48]:
def risk_parity_weight(cov_mat):    
    sigma = np.matrix(cov_mat.values)

    def params(x):
        tmp = (sigma * np.matrix(x).T).A1
        risk = x * tmp
        delta_risk = [sum((i - risk)**2) for i in risk]
        return sum(delta_risk)

    x0 = np.ones(sigma.shape[0]) / sigma.shape[0]  
    bnds = tuple((0,None) for x in x0)
    cons = ({'type':'eq', 'fun': lambda x: sum(x) - 1})
    options={'disp':False, 'maxiter':1000, 'ftol':1e-20}

    res = minimize(params, x0, bounds=bnds, constraints=cons, method='SLSQP', options=options)

    res['success'] == False
    wts = pd.Series(index=cov_mat.index, data=res['x'])
    return wts / wts.sum() * 1.0

In [53]:
test=pd.read_csv("test.csv")
test.set_index("Date")
cov = test.cov()
wts = risk_parity_weight(cov)
wts

Stars3     0.303979
Stars4     0.332032
Starts5    0.363990
dtype: float64

In [55]:
var = (wts.T).dot(cov).dot(wts)
wts*(cov.dot(wts))

Stars3     0.000556
Stars4     0.000556
Starts5    0.000556
dtype: float64